In [1]:
import pandas as pd

In [2]:
mRNA = pd.read_csv('RNAseq6803csv.csv')
mRNA

,location,TU ID,TU start,TU end,Strand,TU type,alternative starts,Internal starts,5'UTR,3'UTR,...,Stat. p.,Max cond.,Max reads,Average,Max/Avg,UEF,Sense tags,Annotation,Antisense TUs,Antisense tags
0,Chr,TU1,105,6,-,a,NaN,NaN,NaN,NaN,...,180,-N,678,217.1,3.12,2.83,NaN,NaN,NaN,slr0611-2
1,Chr,TU2,577,990,+,gai,NaN,NaN,225.0,NaN,...,45,-Fe,938,478.3,1.96,1.07,slr0612,probable pseudouridine synthase,TU3,NaN
2,Chr,TU3,909,810,-,a,NaN,NaN,NaN,NaN,...,45,-N,527,231.5,2.28,1.42,NaN,NaN,TU2,slr0612
3,Chr,TU4,1532,2150,+,ga,NaN,NaN,45.0,52.0,...,1261,15C,3499,2404.2,1.46,1.01,slr0613,hypothetical protein,TU5,sll0558
4,Chr,TU6,2151,2701,+,a,NaN,NaN,NaN,NaN,...,360,-N,2334,1018.6,2.29,1.56,NaN,NaN,TU5,sll0558
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4086,pSYSX,TU6100,102703,102804,+,n,NaN,NaN,NaN,NaN,...,45,-N,828,156.8,5.28,4.55,NaN,NaN,NaN,NaN
4087,pSYSX,TU6101,104362,103451,-,ga,NaN,NaN,35.0,313.0,...,225,-Fe,2207,1042.4,2.12,1.07,sll6109,putative integrase/recombinase,NaN,slr6108
4088,pSYSX,TU6103,104534,104633,+,an,NaN,NaN,NaN,NaN,...,90,-N,1581,301.2,5.25,3.80,NaN,NaN,TU6102,NaN
4089,pSYSX,TU6102,104783,104363,-,an,NaN,NaN,NaN,NaN,...,0,-N,1167,265.3,4.40,3.35,NaN,NaN,TU6103,NaN


In [3]:
mRNA['Strand'].unique()

array(['-', '+'], dtype=object)

In [4]:
mRNA = mRNA[['location', 'Strand', 'TU start', 'TU end', 'Exp. p.']]

In [5]:
mRNA = mRNA[mRNA['location']=='Chr']
mRNA

,location,Strand,TU start,TU end,Exp. p.
0,Chr,-,105,6,106
1,Chr,+,577,990,643
2,Chr,-,909,810,372
3,Chr,+,1532,2150,834
4,Chr,+,2151,2701,1496
...,...,...,...,...,...
3707,Chr,+,3569623,3570366,3711
3708,Chr,+,3570367,3572241,4417
3709,Chr,+,3572695,3572796,46
3710,Chr,-,3573200,3572260,6568


In [ ]:
from Bio import SeqIO
from Bio.Seq import Seq

In [ ]:
genome_sequences = SeqIO.read('whole_genome_pcc6803.fasta', 'fasta').seq

In [ ]:
genome_sequences

In [ ]:
# Function to get the promoter sequence
def get_promoter_sequence(row, genome_sequences):
    tss = row['TU start']
    strand = row['Strand']
    
    full_sequence = genome_sequences
    
    if strand == '+':
        start = max(0, tss - 101) #(-) 생기는거 방지
        end = tss-1
    else:  # strand == '-'
        start = tss+1
        end = min(len(full_sequence), tss + 101)
    
    promoter_seq = full_sequence[start:end]
    
    if strand == '-':
        promoter_seq = promoter_seq.reverse_complement()
    
    return str(promoter_seq)

# Apply the function to each row in the dataframe
mRNA['Promoter_Sequence'] = mRNA.apply(get_promoter_sequence, axis=1, genome_sequences=genome_sequences)

# Save the result to a new CSV file
mRNA.to_csv('PCC6803_promoter_sequences.csv', index=False)


In [ ]:
pcc7942_train = pcc7942[['Promoter Sequence', 'mean']]
pcc7942_train = pcc7942_train.rename(columns={'Promoter Sequence': 'Promoter', 'mean': 'Reads'})
pcc7942_train

In [ ]:
pcc7942_train.to_csv('PCC7942_Promoter and reads 100bp.csv', index=False)

In [ ]:
len(pcc7942_train['Promoter'][100])

In [ ]:
## 다시다시.. 

In [6]:
trlist = []
comp_dic = {'A':'T','G':'C','C':'G','T':'A'}
prolen = 100
numpydf = mRNA.to_numpy()
print(numpydf[:10])


[['Chr' '-' 105 6 106]
 ['Chr' '+' 577 990 643]
 ['Chr' '-' 909 810 372]
 ['Chr' '+' 1532 2150 834]
 ['Chr' '+' 2151 2701 1496]
 ['Chr' '-' 2900 2061 802]
 ['Chr' '-' 3182 2955 1873]
 ['Chr' '-' 4349 3183 48480]
 ['Chr' '-' 6656 4350 2003]
 ['Chr' '-' 6761 6657 1742]]


In [7]:
pccgene = []
pccseq = ''

data = open('Whole_genome_of_synechocystis-Copy1.txt','r')
lines = data.readlines()
line = [line.strip() for line in lines]
pccseq = ''.join(line[1:])
data.close()
pccseq = pccseq.upper()
print(pccseq[:500])

GGCGCGCCATCGCCGGCTGGCGGAAATTACCGAAATGATCCACACCGCTAGTTTGGTCCACGATGACGTGGTGGATGAGGCGGATCTGCGGCGGAATGTGCCCACGGTGAATAGTTTATTTGACAATCGGGTGGCAGTGTTAGCGGGGGATTTCCTCTTTGCCCAATCTTCTTGGTATTTGGCTAACTTAGATAATTTGGAGGTGGTGAAATTATTATCGGAGGTAATTCGGGACTTTGCGGAGGGGGAAATTTTACAGAGCATCAATCGTTTTGACACCGACACAGATTTAGAAACCTATTTGGAAAAAAGCTATTTTAAAACCGCCTCTCTCATTGCCAACAGTGCCAAGGCAGCGGGGGTTTTGAGCGATGCGCCCCGGGATGTGTGTGATCATCTTTACGAATATGGTAAACATTTGGGGTTAGCGTTCCAGATTGTGGACGATATTTTAGATTTCACTTCCCCCACGGAGGTTTTGGGGAAACCGGCCGGGTCAG


In [8]:
for i in numpydf:
    # chromosomal DNA
    if i[0] == 'Chr':
        #positive strands
        if i[1] == '+':
            # pccseq[i[2](TSS) - (prolen+1) (TSS로부터 100bp upstream) : TSS-1] - promoter sequence indexing, i[19]: RNA reads
            trlist.append((pccseq[i[2]-(prolen+1):i[2]-1],i[4]))

        # negative strands
        else:
            compseq = ''
            seq = pccseq[i[2]:i[2]+prolen]
            for s in seq:
                compseq +=comp_dic[s]
            #역방향으로 complementary seq promoter 추출
            revcomp_seq = compseq[::-1]
            trlist.append((revcomp_seq, i[4]))
    else:
        break

In [29]:
trlist

[('CACCTCCAAATTATCTAAGTTAGCCAAATACCAAGAAGATTGGGCAAAGAGGAAATCCCCCGCTAACACTGCCACCCGATTGTCAAATAAACTATTCACC',
  106),
 ('TTTTGGGGAAACCGGCCGGGTCAGATTTAATCAGCGGCAACATCACCGCCCCAGCCCTATTTGCCATGGAAAAATATCCCCTACTTGGTAAATTAATTGA',
  643),
 ('CCAAACGATGCTGAAGTTTACCGTTGCTGGTGAGCAGCAATAGTCCTTCGCTATCTTGATCCAAACGCCCCACGGGATATAAATCTGGCAAATTAATATA',
  372),
 ('CAGCAACTCTTAACGGGAAATCCCAATGGTCCCTGGCAGAAAAAATTTGGCGATCGCCGTTGATCAAGGCTCCCTCCCCCCAGGGCATTAAAATAGGAAC',
  834),
 ('CCTGAATATCTCCAGGGTTATACCGCCCCCGATGAAGCTTTTGTTTGAACTCCATCTTGAGCGGTAATGACTTCCCCGAAGAAGTTATAAACTTGTAGGC',
  1496),
 ('TGAGAATTTTGCCAAGTTTAGGTAAAATTTTTGGGGCATTTTTCCTGGCTGAAAATTTTTTGGCGGTGTGTTCGCCGCAATCATGGCAGATAATAGGGTT',
  802),
 ('ACCCATTTTTGCTTCCAACGGCTGGCAATTTATCAAACTGTATTTGATGAAGCCCATTGCAGTGGATCAGTTGGCTGGAGCTGTGCGCTAAGGTTGTCTT',
  1873),
 ('TACTGCAAAAAGTTCCGTTTGCAACAAAACACGTAACAGTTTGTAACAGTTGCCCCCCCTGACTAGAAACCCCGTTTCAACCCTTGTTACACTAGGCAAT',
  48480),
 ('CCAGCAAATATCCACTGGAACGCTAAAGCCGCAATCGTTGCTCACCATGGCGATCGCCAGAAGGCCTGCATCTGGGCAAGG

In [20]:
final_list = []

In [21]:
for i in range(len(trlist)):
    final_list.append(trlist[i][0])

In [26]:
df = pd.DataFrame(final_list)
df

,0
0,CACCTCCAAATTATCTAAGTTAGCCAAATACCAAGAAGATTGGGCA...
1,TTTTGGGGAAACCGGCCGGGTCAGATTTAATCAGCGGCAACATCAC...
2,CCAAACGATGCTGAAGTTTACCGTTGCTGGTGAGCAGCAATAGTCC...
3,CAGCAACTCTTAACGGGAAATCCCAATGGTCCCTGGCAGAAAAAAT...
4,CCTGAATATCTCCAGGGTTATACCGCCCCCGATGAAGCTTTTGTTT...
...,...
3707,GAACCAAGTCGATTTGGGTTTATGAAAGCCAGAGGACAGGGCGGGG...
3708,ACAACGTTGATCTACGGGCCGTTTATCGCAAGCTTGGCGATCGTCG...
3709,TGCTTAGTTGAATTAGCTATAAACTAAATCAGTCAATTAGTCAGCT...
3710,GGGGGCAAACAGGGAGGTAGTGGAGATCATCGGCTGGCCAAACTTG...


In [28]:
df.to_csv('final.csv', index = False)